In [17]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import cv2
import cv2
import numpy as np
from skimage.metrics import mean_squared_error as mse
import time

In [18]:
def split_image_into_patches(image, patch_size): # Splitting images into patches 
    patches = []
    height, width = image.shape[:2]
    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            patches.append(patch)
    return patches

def apply_edge_detection(patches): # Applying canny edge detection
    edges = []
    for patch in patches:
        gray_patch = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
        edge = cv2.Canny(gray_patch, 50, 150)
        edges.append(edge)
    return edges

def count_edges(patches): # Counting edges
    edges = []
    for patch in patches:
        edges.append(np.count_nonzero(patch))
    return edges

def calculate_differences(features): # Calculating absolute difference 
    differences = []
    num_patches = len(features)
    for i in range(num_patches):
        for j in range(num_patches):
            difference = abs(features[i] - features[j])
            differences.append({'Patch1': i, 'Patch2': j, 'Difference': difference})
    return pd.DataFrame(differences)

def display_image_and_patches(image, patches, patch_size): # Function to display image and it patches
    height, width = image.shape[:2]
    # Create an empty image with lines separating patches
    output_image = np.copy(image)
    for y in range(0, height, patch_size):
        cv2.line(output_image, (0, y), (width, y), (0, 255, 0), 1)
    for x in range(0, width, patch_size):
        cv2.line(output_image, (x, 0), (x, height), (0, 255, 0), 1)
    
    # Display the original image and lines separating patches
    cv2.imshow("Original Image with Patches", output_image)
    
    # Display each individual patch with edge detection applied
    for i, patch in enumerate(patches):
        # Apply edge detection to the patch
        edges = apply_edge_detection(patch)
        # Display the patch with edges
        cv2.imshow(f"Patch {i} with Edges", np.hstack((patch, edges)))
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return 

#Reading Images
image1 = cv2.imread(r"C:\Users\DELL\Desktop\img1.jpg")
image2 = cv2.imread(r"C:\Users\DELL\Desktop\img2.jpg")

#Resizing image into size of other image
target_height, target_width = image2.shape[:2]
image1_new = cv2.resize(image1, (target_width, target_height))
cv2.imwrite('image1_new.jpg', image1_new)

# Splitting images into patches
patch_size = 100
patches_img1 = split_image_into_patches(image1_new, patch_size)
patches_img2 = split_image_into_patches(image2, patch_size)

#Displaying Images
#display_image_and_patches(image1, patches_img1, patch_size)
#display_image_and_patches(image2, patches_img2, patch_size)

# Applying edge detection
img1_edges = apply_edge_detection(patches_img1)
img2_edges = apply_edge_detection(patches_img2)


#Counting Edges
img1_no_edges = count_edges(img1_edges)
img2_no_edges = count_edges(img2_edges)
img1

In [19]:
# Constructing graphs
differences_df_img1 = calculate_differences(img1_no_edges)
differences_df_img2 = calculate_differences(img2_no_edges)
matrix1 =  differences_df_img1['Difference'].values
matrix2 =  differences_df_img2['Difference'].values
m1= matrix1.reshape(8,8)
m2= matrix2.reshape(8,8)
m2



array([[   0, 1391,  349, 1048,  671, 1017, 2393, 2453],
       [1391,    0, 1740,  343,  720,  374, 1002, 1062],
       [ 349, 1740,    0, 1397, 1020, 1366, 2742, 2802],
       [1048,  343, 1397,    0,  377,   31, 1345, 1405],
       [ 671,  720, 1020,  377,    0,  346, 1722, 1782],
       [1017,  374, 1366,   31,  346,    0, 1376, 1436],
       [2393, 1002, 2742, 1345, 1722, 1376,    0,   60],
       [2453, 1062, 2802, 1405, 1782, 1436,   60,    0]], dtype=int64)

In [20]:
# Applying Hungarian Algorithm
# Calculate differences and normalize
diff_matrix = abs(m1 - m2)
normalized_diff_matrix = diff_matrix / diff_matrix.max()
# Invert for similarity
cost_matrix = 1 - normalized_diff_matrix
# Apply the Hungarian algorithm
row_ind, col_ind = linear_sum_assignment(cost_matrix)
# Calculate the similarity score 
similarity_score = cost_matrix[row_ind, col_ind].sum() 

print("Similarity Score:", similarity_score)

Similarity Score: 3.2143497757847532


In [21]:
import numpy as np
from scipy.optimize import linear_sum_assignment
import time

# Assuming m1 and m2 are your matrices

# Start time measurement
start_time = time.time()

# Calculate differences and normalize
diff_matrix = np.abs(m1 - m2)
normalized_diff_matrix = diff_matrix / diff_matrix.max()

# Invert for similarity
cost_matrix = 1 - normalized_diff_matrix

# Apply the Hungarian algorithm
row_ind, col_ind = linear_sum_assignment(cost_matrix)

# Calculate the similarity score
similarity_score = cost_matrix[row_ind, col_ind].sum()

# End time measurement
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Similarity Score:", similarity_score)
print("Time taken:", elapsed_time, "seconds")


Similarity Score: 3.2143497757847532
Time taken: 0.0009975433349609375 seconds


In [24]:
# Convert images to grayscale (SSIM requires grayscale images)
image1_gray = cv2.cvtColor(image1_new, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = ssim(image1_gray, image2_gray)
# Measure time for SSIM calculation
start_time = time.time()
ssim_score = ssim(image1_gray, image2_gray)
ssim_time = time.time() - start_time


# Calculate RMSE
rmse_score = np.sqrt(mse(image1_gray, image2_gray))
# Measure time for RMSE calculation
start_time = time.time()
rmse_score = np.sqrt(mse(image1_gray, image2_gray))
rmse_time = time.time() - start_time

# Calculate PSNR
psnr_score = psnr(image1_gray, image2_gray)
# ISSM is combination ssim and psnr score
issm_score = (ssim_score + psnr_score) / 2
# Measure time for PSNR calculation
start_time = time.time()
psnr_score = psnr(image1_gray, image2_gray)
psnr_time = time.time() - start_time


# Print the results
print("Structural Similarity Index (SSIM):", ssim_score)
print("Time taken for SSIM calculation:", ssim_time, "seconds")
print("Root Mean Square Error (RMSE):", rmse_score)
print("Time taken for RMSE calculation:", rmse_time, "seconds")
print("Peak Signal-to-Noise Ratio (PSNR):", psnr_score)
print("Time taken for PSNR calculation:", psnr_time, "seconds")
print("Image Structural Similarity Measure (ISSM):", issm_score)


Structural Similarity Index (SSIM): 0.0621739025638859
Time taken for SSIM calculation: 0.011968135833740234 seconds
Root Mean Square Error (RMSE): 57.264719688865185
Time taken for RMSE calculation: 0.000997304916381836 seconds
Peak Signal-to-Noise Ratio (PSNR): 12.973060824957098
Time taken for PSNR calculation: 0.0009987354278564453 seconds
Image Structural Similarity Measure (ISSM): 6.517617363760492
